## MFBO example

In [ ]:
#!/usr/bin/env python
# coding: utf-8
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys

sys.path.insert(0, '../')
from pyTrajectoryUtils.pyTrajectoryUtils.minSnapTrajectory import *
from pyTrajectoryUtils.pyTrajectoryUtils.utils import *

yaml_name = 'mfbo_example'
sample_name_ = 'half_space_figure8'
drone_model = "STMCFB"

traj_tool = TrajectoryTools(N_POINTS=40)
poly = MinSnapTrajectory(N_POINTS=40, drone_model=drone_model, yaw_mode=3)
points, t_set = get_waypoints(yaml_name, sample_name_, flag_t_set=True)

########################################################
# Min snap trajectory
# Input
#   alpha_scale - scale overall time with alpha_scale value
#   yaw_mode
#     0: set all yaw to 0, 
#     1: set trajectory yaw forward, 
#     2: set trajectory yaw as the reference yaw (points[:,3]) (cos/sin format) 
#     3: set trajectory yaw as the reference yaw (directly use yaw ref)
#   flag_loop - whether the trajectory has same initial and final position and attitude
#   deg_init_min, deg_init_max - the initial state is set to 0 from deg_init_min - th derivative to deg_init_max - th derivative
#       (e.g. deg_init_min=0, deg_init_max=2: initial position is set to points[0,:] and initial velocity, acceleration is set to zero)
#   deg_end_min, deg_end_max - the final state is set to 0 from deg_end_min - th derivative to deg_end_max - th derivative
#   ded_init_yaw_min=0, deg_init_yaw_max - set the initial yaw value and it's derivative to zero
#   ded_end_yaw_min=0, deg_end_yaw_max - set the fianl yaw value and it's derivative to zero
#   flag_rand_init - whether to use random initialization on optimization process. 
#       (randomly generate set of time allocation and select the on minimze the overall snap)
#   flag_numpy_opt - whether to use numpy-based gradient descent to find initial value (often improve the final result).
# Output
#   t_set - time allocation between waypoints
#   d_ordered - position and derivatives on each waypoint
#   d_ordered - yaw value and angular speed, acceleration on each waypoint
t_set_new = t_set
t_set_new, d_ordered, d_ordered_yaw = poly.get_min_snap_traj(
    points, alpha_scale=1.0, flag_loop=False, yaw_mode=3, \
    deg_init_min=0, deg_init_max=4, \
    deg_end_min=0, deg_end_max=4, \
    deg_init_yaw_min=0, deg_init_yaw_max=2, \
    deg_end_yaw_min=0, deg_end_yaw_max=2, \
    flag_rand_init=False, flag_numpy_opt=False, flag_opt_alpha=True)
print("t_set = np.array([{}])".format(','.join([str(x) for x in t_set_new])))
print("########################################################")

########################################################
# Change trajectory speed
# multiply alpha_set with t_set and generate new d_orderd (derivates on each waypoints) and d_ordered_yaw
t_set_new, d_ordered, d_ordered_yaw = poly.update_traj( 
    points=points, \
    t_set=t_set_new, \
    deg_init_min=0, deg_init_max=4, \
    deg_end_min=0, deg_end_max=4, \
    deg_init_yaw_min=0, deg_init_yaw_max=2, \
    deg_end_yaw_min=0, deg_end_yaw_max=2, \
    alpha_set=np.ones_like(t_set_new), \
    yaw_mode=3)
max_spd = traj_tool.get_max_speed(t_set_new, d_ordered, flag_print=False)
print("max_spd: {:.2f} m/s".format(max_spd))


########################################################
# Trajectory tools - plot and save trajectory
fig, axs = plt.subplots(1,1)
fig.set_size_inches((6,4))
traj_tool.plot_trajectory_2D_single(axs, t_set_new, d_ordered, d_ordered_yaw, direct_yaw=True)
plt.show()

########################################################
# Low-fidelity evaluation: Check whether the reference motor speed exceed the maximum motor speed
print("Sanity check (ref. motor speed) result : {}". \
    format(poly.sanity_check(t_set_new, d_ordered, d_ordered_yaw, flag_parallel=True, direct_yaw=True)))
print("########################################################")


########################################################
# High-fidelity evaluation: Run simulation with indi controller
debug_array = poly.sim.run_simulation_from_der( \
    t_set=t_set_new, d_ordered=d_ordered, d_ordered_yaw=d_ordered_yaw, \
    max_pos_err=0.2, min_pos_err=0.1, max_yaw_err=90., min_yaw_err=60.0, freq_ctrl=200, freq_sim=400)
poly.sim.plot_result( \
    debug_array[0], flag_save=False, \
    save_dir="../trajectory/result", save_idx="0", \
    t_set=t_set_new, d_ordered=d_ordered)
print("########################################################")



In [ ]:
import time

N_trial = 20
# Start timer
start_time = time.perf_counter()
for nn in range(N_trial):
    t_set_t, d_ordered_t, d_ordered_yaw_t = poly.update_traj( 
        points=points, \
        t_set=t_set_new, \
        deg_init_min=0, deg_init_max=4, \
        deg_end_min=0, deg_end_max=4, \
        deg_init_yaw_min=0, deg_init_yaw_max=2, \
        deg_end_yaw_min=0, deg_end_yaw_max=2, \
        alpha_set=np.ones_like(t_set_new), \
        yaw_mode=3)
    poly.sanity_check(t_set_t, d_ordered_t, d_ordered_yaw_t, flag_parallel=True, direct_yaw=True)
# End timer
end_time = time.perf_counter()

# Calculate elapsed time
elapsed_time = (end_time - start_time)/N_trial
print("Sta computation time: ", elapsed_time)

start_time = time.perf_counter()
for nn in range(N_trial):
    t_set_t, d_ordered_t, d_ordered_yaw_t = poly.update_traj( 
        points=points, \
        t_set=t_set_new, \
        deg_init_min=0, deg_init_max=4, \
        deg_end_min=0, deg_end_max=4, \
        deg_init_yaw_min=0, deg_init_yaw_max=2, \
        deg_end_yaw_min=0, deg_end_yaw_max=2, \
        alpha_set=np.ones_like(t_set_new), \
        yaw_mode=3)
    debug_array = poly.sim.run_simulation_from_der( \
        t_set=t_set_t, d_ordered=d_ordered_t, d_ordered_yaw=d_ordered_yaw_t, \
        max_pos_err=0.2, min_pos_err=0.1, max_yaw_err=90., min_yaw_err=60.0, freq_ctrl=200, freq_sim=400)
# End timer
end_time = time.perf_counter()

# Calculate elapsed time
elapsed_time = (end_time - start_time)/N_trial
print("Sim computation time: ", elapsed_time)


# Run MFBO

In [ ]:
!python3 ../scripts_mfbo/run_mfbo.py